# Repeat-Until-Success Sample
This sample shows the use of branching on measurement and qubit reuse to implement a Repeat-Until-Success algorithm on Quantinuum hardware targets. It is drawn from a collaboration between Microsoft and Quantinuum research teams, specifically the authors Natalie C. Brown, John Peter Campora III, Ciarán Ryan-Anderson, Dominic Lucchetti, and Alex Chernoguzov from Quantinuum; Stefan Wernli, Adam Paetznick, Martin Roetteler, and Krysta Svore from Microsoft; as well as Bettina Heim and Cassandra Granade.

The program uses RUS and two sub-circuits to perform the single-qubit unitary V<sub>3</sub> = (I + 2iZ)/(√5) as described in detail in [Repeat-Until-Success: Non-deterministic decomposition of single-qubit unitaries](https://arxiv.org/abs/1311.1074) by Adam Paetznik and Krysta Svore.



## 1. Connect to Azure Quantum and set up target

First, we must configure the qsharp module to connect the azure workspace and specify a target. The Quantinuum H1 emulator target `quantinuum.sim.h1-1e` is chosen by default, which performs hardware-modeled noisy simulation. Running the experiments in this notebook against that target will consume approximately 25 HQC.

We also configure the target profile as `Adaptive_RI` to indicate we use the QIR profile with support for mid-circuit measurement, measurement-based control flow, and classical integer computation as part of compilation.

Replace the `subscription_id`, `resource_group`, `name`, and `location` connection parameters with the values for your configured Azure Quantum Workspace, or try copying the Python code for connecting to your workspace from the VS Code Azure Quantum Extension.

In [ ]:
import qsharp

qsharp.init(target_profile=qsharp.TargetProfile.Adaptive_RI)

In [ ]:
import azure.quantum

workspace = azure.quantum.Workspace(
    subscription_id = "SUBSCRIPTION ID",
    resource_group = "RESOURCE GROUP",
    name = "WORKSPACE NAME",
    location = "LOCATION",
)

In [ ]:
target = workspace.get_targets("quantinuum.sim.h1-1e")

## 2. Compile the RUS Program

In [ ]:
%%qsharp
import Std.Math.*;

/// # Summary
/// Example of a Repeat-until-success algorithm implementing a circuit
/// that achieves exp(i*ArcTan(2)*Z), also known as the "V gate"
/// by Paetznick & Svore.
///
/// # References
/// - [ *Adam Paetznick, Krysta M. Svore*,
/// Quantum Information & Computation 14(15 & 16): 1277-1301 (2014)
/// ](https://arxiv.org/abs/1311.1074)
/// For circuit diagram, see file RUS.png.
///
/// # Input
/// ## inputValue
/// Boolean value for input qubit (true maps to One, false maps to Zero)
/// ## inputBasis
/// Pauli basis in which to prepare input qubit
/// ## limit
/// Integer limit to number of repeats of circuit
///
/// # Remarks
/// The program executes a circuit on a "target" qubit using an "auxiliary"
/// and "resource" qubit. The circuit consists of two parts (red and blue
/// in image).
/// The goal is to measure Zero for both the auxiliary and resource qubit.
/// If this succeeds, the program will have effectively applied an
/// Rz(arctan(2)) gate (also known as V_3 gate) on the target qubit.
/// If this fails, the program reruns the circuit up to <limit> times.
operation AllocateQubitsAndApplyRzArcTan2 (
    inputValue : Bool,
    inputBasis : Pauli,
    measurementBasis : Pauli,
    limit : Int
) : Result[] {
    use (auxiliary, resource, target) = (Qubit(), Qubit(), Qubit());

    // Prepare target qubit in a zero- or one-state, based on input value
    if (inputValue) {
        X(target);
    }
    PreparePauliEigenstate(inputBasis, target);

    within {
        H(auxiliary);
        H(resource);
    }
    apply {
        ApplyRzArcTan2(limit, auxiliary, resource, target);

        let rotationAngle = 2.0 * ArcTan(2.0);
        Rz(rotationAngle, target); // Rotate back to initial state
    }

    return [M(auxiliary), M(resource), Measure([measurementBasis], [target])];
}

/// # Summary
/// Apply Rz(arctan(2)) on qubits using repeat until success algorithm.
/// Updated to use for-loop sytle.
operation ApplyRzArcTan2 (
    limit : Int,
    auxiliary : Qubit,
    resource : Qubit,
    target : Qubit
) : Unit {
    mutable (result1, result2) = (false, false);

    // Run Part 1 of the program.
    for _ in 1..limit {
        if not result1 or not result2 {
            set result1 = ApplyAndMeasurePart1(auxiliary, resource) == Zero;

            // We’ll only run Part 2 if Part 1 returns Zero.
            // Otherwise, we’ll skip and rerun Part 1 again.
            if result1 {
                set result2 = ApplyAndMeasurePart2(resource, target) == Zero;

                if not result2 {
                    Z(resource); // Reset resource
                    Adjoint Z(target); // Correct effective Z rotation on target
                }
            } else {
                Z(auxiliary);
                Reset(resource);
                H(resource);
            }
        }
    }
}

/// # Summary
/// Apply part 1 of RUS circuit and measure
/// auxiliary qubit in Pauli X basis
operation ApplyAndMeasurePart1(auxiliary : Qubit, resource : Qubit) : Result {
    within {
        T(auxiliary);
    }
    apply {
        CNOT(resource, auxiliary);
    }
    return Measure([PauliX], [auxiliary]);
}

/// # Summary
/// Apply part 2 of RUS circuit and measure
/// resource qubit in Pauli X basis
operation ApplyAndMeasurePart2(resource : Qubit, target : Qubit) : Result {
    T(target);
    Z(target);
    CNOT(target, resource);
    T(resource);
    return Measure([PauliX], [resource]);
}

operation PreparePauliEigenstate(basis : Pauli, qubit : Qubit) : Unit {
    if (basis == PauliX) {
        H(qubit);
    } elif (basis == PauliY) {
        H(qubit);
        S(qubit);
    }
}

## 3. Run the Experiment
We can now submit and wait on each of three experiments using different repeat limits, displaying the resulting histograms and saving them into variables for later analysis. We should see the percentage of resulting successes increase as more repeats are allowed to execute.

In [ ]:
job0 = target.submit(qsharp.compile("AllocateQubitsAndApplyRzArcTan2(false, PauliX, PauliX, 0)"), shots=100)
job0.wait_until_completed()
results0 = job0.get_results()

In [ ]:
job1 = target.submit(qsharp.compile("AllocateQubitsAndApplyRzArcTan2(false, PauliX, PauliX, 1)"), shots=100)
job1.wait_until_completed()
results1 = job1.get_results()

In [ ]:
job2 = target.submit(qsharp.compile("AllocateQubitsAndApplyRzArcTan2(false, PauliX, PauliX, 2)"), shots=100)
job2.wait_until_completed()
results2 = job2.get_results()

## 4. Plot the Results
Finally, we plot the results. When run against the emulator or hardware, each successive experiment should show increasing probability of success, indicating that increasing the allowed runs of RUS further refines the effectiveness of the program.

In [ ]:
from matplotlib import pyplot

success = "[0, 0, 0]"
xticks = [0, 1, 2]
pyplot.title("Probability Success")
pyplot.plot(xticks, [results0[success], results1[success], results2[success]])
pyplot.xticks([0, 1, 2])
pyplot.xlabel("Repeat Limit")
pyplot.ylabel("Probability")


pyplot.show()